[Reference](https://ai.plainenglish.io/introducing-fastmcp-v2-the-pythonic-way-to-build-secure-mcp-servers-and-clients-for-ai-fcc09b84771a)

In [2]:
!pip install fastmcp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.0/312.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.4/224.4 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3


# Server Setup (mcp_server.py)

In [3]:
from fastmcp import FastMCP
from fastmcp.prompts.prompt import PromptMessage, TextContent

# Create a basic server instance
mcp = FastMCP(name="MyMCPServer")

# Tool
@mcp.tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

# Resource
@mcp.resource("resource://greeting")
def get_greeting() -> str:
    """Provides a simple greeting message."""
    return "Hello from FastMCP Resources!"

# Resource Template
@mcp.resource("data://{name}/greeting")
def get_greeting_by_name(name: str) -> str:
    """Provides a greeting message for a specific name."""
    return f"Hello {name} from FastMCP Resources!"

# Basic prompt returning a string (converted to user message automatically)
@mcp.prompt
def ask_about_topic(topic: str) -> str:
    """Generates a user message asking for an explanation of a topic."""
    return f"Can you please explain the concept of '{topic}'?"

# Prompt returning a specific message type
@mcp.prompt
def generate_code_request(language: str, task_description: str) -> PromptMessage:
    """Generates a user message requesting code generation."""
    content = f"Write a {language} function that performs the following task: {task_description}"
    return PromptMessage(role="user", content=TextContent(type="text", text=content))

if __name__ == "__main__":
    # Start server with STDIO
    mcp.run()
    # Start server with HTTP on port 8000
    # mcp.run(transport="http", host="127.0.0.1", port=8000)

RuntimeError: Already running asyncio in this thread

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


# Client Interaction (mcp_client.py)

In [ ]:
from fastmcp import Client

# connect Local server (STDIO)
client = Client("mcp_server.py")

# connect HTTP server
# client = Client("http://127.0.0.1:8000/mcp")

async def call_mcp():
    async with client:
        # Basic server interaction
        await client.ping()

        # List available operations
        tools = await client.list_tools()
        resources = await client.list_resources()
        resource_templates= await client.list_resource_templates()
        prompts = await client.list_prompts()

        print("\n===============================Tools===============================\n")
        print("\n".join([tool.name for tool in tools]))
        print("\n===============================Resources===============================\n")
        print("\n".join([resource.name for resource in resources]))
        print("\n===============================Resource Templates===============================\n")
        print("\n".join([resource_template.name for resource_template in resource_templates]))
        print("\n===============================Prompts===============================\n")
        print("\n".join([prompt.name for prompt in prompts]))

        # Call tools
        result = await client.call_tool("add", {"a": 1, "b": 2})
        print("\n===============================Tool Result===============================\n")
        print(result.content[0].text)

        # Call resources
        result = await client.read_resource("resource://greeting")
        print("\n===============================Resource Result===============================\n")
        print(result[0].text)

        # Call prompts
        messages = await client.get_prompt("ask_about_topic", {"topic": "AI"})
        print("\n===============================Prompt Result===============================\n")
        print(messages.messages[0].content.text)

if __name__ == "__main__":
    import asyncio
    asyncio.run(call_mcp())

# Advanced Server Setup (advanced_mcp_features_server.py)

In [5]:
from fastmcp import FastMCP, Context
from fastmcp.server.dependencies import get_context
from dataclasses import dataclass
import asyncio

# Create a basic server instance
mcp = FastMCP(name="MyAdvancedMCPServer")


#-----------------------Context-------------------------------
@mcp.tool
async def process_file(file_uri: str, ctx: Context) -> str:
    """Processes a file, using context for logging and resource access."""
    # Context is available as the ctx parameter
    return "Processed file"

# Utility function that needs context but doesn't receive it as a parameter
async def process_data(data: list[float]) -> dict:
    # Get the active context - only works when called within a request
    ctx = get_context()
    await ctx.info(f"Processing {len(data)} data points")

@mcp.tool
async def analyze_dataset(dataset_name: str) -> dict:
    # Call utility function that uses context internally

    # load data from file
    # data = load_data(dataset_name)

    data = [1,2,3,4,5,6,7,8,9,10]
    await process_data(data)

    return {"processed": len(data), "results": data}

#-----------------------Elicitation---------------------------
@dataclass
class UserInfo:
    name: str
    age: int

@mcp.tool
async def collect_user_info(ctx: Context) -> str:
    """Collect user information through interactive prompts."""
    result = await ctx.elicit(
        message="Please provide your information",
        response_type=UserInfo
    )

    if result.action == "accept":
        user = result.data
        return f"Hello {user.name}, you are {user.age} years old"
    elif result.action == "decline":
        return "Information not provided"
    else:  # cancel
        return "Operation cancelled"

#----------------------Progress Reporting-------------------------------
@mcp.tool
async def process_items(items: list[str], ctx: Context) -> dict:
    """Process a list of items with progress updates."""
    total = len(items)
    results = []

    for i, item in enumerate(items):
        # Report progress as we process each item [total is optional]
        await ctx.report_progress(progress=i, total=total)

        # Simulate processing time
        await asyncio.sleep(1)
        results.append(item.upper())

    # Report 100% completion
    await ctx.report_progress(progress=total, total=total)

    return {"processed": len(results), "results": results}

#----------------------LLM Sampling---------------------------

@mcp.tool
async def analyze_sentiment(text: str, ctx: Context) -> dict:
    """Analyze the sentiment of text using the client's LLM."""
    prompt = f"""Analyze the sentiment of the following text as positive, negative, or neutral.
    Just output a single word - 'positive', 'negative', or 'neutral'.

    Text to analyze: {text}"""

    # Request LLM analysis
    response = await ctx.sample(prompt)

    # from fastmcp.client.sampling import SamplingMessage
    # messages = [
    #     SamplingMessage(role="user", content=f"I have this data: {context_data}"),
    #     SamplingMessage(role="assistant", content="I can see your data. What would you like me to analyze?"),
    #     SamplingMessage(role="user", content=user_query)
    # ]

    #response = await ctx.sample(
    #     messages=messages,
    #     system_prompt="You are an expert Python programmer. Provide concise, working code examples without explanations.",
    #     model_preferences="claude-3-sonnet",  # Prefer a specific model
    #     include_context="thisServer",  # Use the server's context
    #     temperature=0.7,
    #     max_tokens=300
    # )

    # Process the LLM's response
    sentiment = response.text.strip().lower()

    # Map to standard sentiment values
    if "positive" in sentiment:
        sentiment = "positive"
    elif "negative" in sentiment:
        sentiment = "negative"
    else:
        sentiment = "neutral"

    return {"text": text, "sentiment": sentiment}


if __name__ == "__main__":
    # Start server with HTTP on port 8000
    mcp.run(transport="http", host="127.0.0.1", port=8000)

# Multi-Server Clients and Handlers

## Multiserver Client (multiserver_mcp_client.py):

In [6]:
from fastmcp import Client
from fastmcp.client.elicitation import ElicitResult
from fastmcp.client.logging import LogMessage
from fastmcp.client.sampling import (
    SamplingMessage,
    SamplingParams,
    RequestContext,
)


# Local server (STDIO)
# client = Client("advanced_mcp_features_server.py")

# HTTP server
# client = Client("http://127.0.0.1:8000/mcp")

# JSON config (multiple servers)
# Tools are namespaced by server name eg: server_name_tool_name [my_server_add]
# Resources are namespaced by server name eg: resource://server_name/resource_name [resource://my_server/greeting]
config = {
    "mcpServers": {
        "my_advanced_server": {
            "url": "http://127.0.0.1:8000/mcp",
            "transport": "http"
        },
        "my_server": {
            "command": "python",
            "args": ["./mcp_server.py"],
            "env": {"LOG_LEVEL": "INFO"}
        }
    }
}


async def elicitation_handler(message: str, response_type: type, params, context):
    # Present the message to the user and collect input
    print(message)
    name = input("Enter name: ")
    age = input("Enter age: ")

    if name == "" or age == "":
        return ElicitResult(action="decline")

    # Create response using the provided dataclass type
    # FastMCP converted the JSON schema to this Python type for you
    response_data = response_type(name=name, age=age)

    # You can return data directly - FastMCP will implicitly accept the elicitation
    return response_data

    # Or explicitly return an ElicitResult for more control
    # return ElicitResult(action="accept", content=response_data)


async def log_handler(message: LogMessage):
    """
    Handles incoming logs from the MCP server and forwards them
    to the standard Python logging system.
    """
    msg = message.data.get('msg')
    extra = message.data.get('extra')
    level = message.level

    print("======Log Data from handler======")
    print(msg)
    print(extra)
    print(level)
    print("=================================")

    # Convert the MCP log level to a Python log level
    # level = LOGGING_LEVEL_MAP.get(message.level.upper(), logging.INFO)

    # Log the message using the standard logging library
    # logger.log(level, msg, extra=extra)

async def progress_handler(
    progress: float,
    total: float | None,
    message: str | None
) -> None:
    if total is not None:
        percentage = (progress / total) * 100
        print(f"Progress: {percentage:.1f}% - {message or ''}")
    else:
        print(f"Progress: {progress} - {message or ''}")

async def sampling_handler(
    messages: list[SamplingMessage],
    params: SamplingParams,
    context: RequestContext
) -> str:
    print("======Sampling Data from handler======")
    print(messages[0].content.text)
    print("=================================")
    # Your LLM integration logic here
    # Extract text from messages and generate a response
    return "neutral"

client = Client(
    config,
    elicitation_handler=elicitation_handler,
    log_handler=log_handler,
    progress_handler=progress_handler,
    sampling_handler=sampling_handler
    )

async def call_mcp():
    async with client:
        # Basic server interaction
        await client.ping()

        # List available operations
        tools = await client.list_tools()
        resources = await client.list_resources()
        resource_templates= await client.list_resource_templates()
        prompts = await client.list_prompts()

        print("\n===============================Tools===============================\n")
        print("\n".join([tool.name for tool in tools]))
        print("\n===============================Resources===============================\n")
        print("\n".join([resource.name for resource in resources]))
        print("\n===============================Resource Templates===============================\n")
        print("\n".join([resource_template.name for resource_template in resource_templates]))
        print("\n===============================Prompts===============================\n")
        print("\n".join([prompt.name for prompt in prompts]))

        # Call tools
        result = await client.call_tool("my_server_add", {"a": 1, "b": 2})
        print("\n===============================Tool Result===============================\n")
        print(result.content[0].text)

        # Call resources
        result = await client.read_resource("resource://my_server/greeting")
        print("\n===============================Resource Result===============================\n")
        print(result[0].text)

        # Call prompts
        messages = await client.get_prompt("my_server_ask_about_topic", {"topic": "AI"})
        print("\n===============================Prompt Result===============================\n")
        print(messages.messages[0].content.text)

        # Call advanced server tools with context
        result = await client.call_tool("my_advanced_server_process_file", {"file_uri": "file://test.txt"})
        print("\n===============================Advanced Server Tool with context Result===============================\n")
        print(result.content[0].text)

        # Call advanced server tools with elicitation
        print("\n===============================Advanced Server Tool with elicitation Result===============================\n")
        result = await client.call_tool("my_advanced_server_collect_user_info")
        print(result.content[0].text)

        # Call advanced server tools with logging
        print("\n===============================Advanced Server Tool with logging Result===============================\n")
        result = await client.call_tool("my_advanced_server_analyze_dataset", {"dataset_name": "test.txt"})
        print(result.content[0].text)

        # Call advanced server tools with progress reporting
        print("\n===============================Advanced Server Tool with progress reporting Result===============================\n")
        result = await client.call_tool("my_advanced_server_process_items", {"items": ["item1", "item2", "item3"]})
        print(result.content[0].text)

        #Call advanced server tools with LLM sampling
        print("\n===============================Advanced Server Tool with LLM sampling Result===============================\n")
        result = await client.call_tool("my_advanced_server_analyze_sentiment", {"text": "AI is the future of technology"})
        print(result.content[0].text)

if __name__ == "__main__":
    import asyncio
    asyncio.run(call_mcp())